In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import re

import matplotlib.cm as cm
import matplotlib.pyplot as plt
# df.sample(3, random_state=42)

In [2]:
df = pd.read_csv('raw/MPS_LSOA_Level_Crime_Historic.csv')
df.shape

(121672, 136)

In [3]:
msoa_nms = pd.read_csv('raw/Output_Area_to_LSOA_to_MSOA_to_Local_Authority_District_(December_2017)_Lookup_with_Area_Classifications_in_Great_Britain.csv')
msoa_nms.shape

(227759, 18)

In [4]:
df['Major Category'].unique()

array(['Violence Against the Person', 'Burglary', 'Criminal Damage',
       'Drugs', 'Theft and Handling', 'Other Notifiable Offences',
       'Robbery'], dtype=object)

In [5]:
monthcols = [c for c in df.columns.values if re.search('\d{6}', c)]

month_agg = {}

for m in monthcols:
    month_agg[m] = 'sum'

In [6]:
msoa_nms.drop(columns=['OA11CD', 'OAC11CD', 'OAC11NM', 'SOAC11CD',
       'SOAC11NM', 'LAD17CD', 'LAD17NM', 'LACCD',
       'LACNM', 'RGN11CD', 'RGN11NM', 'CTRY11CD', 'CTRY11NM', 'FID'], inplace=True)

In [7]:
msoa_nms.drop_duplicates(subset=['LSOA11CD', 'LSOA11NM'], inplace=True)

In [8]:
msoa_nms.shape

(41729, 4)

In [9]:
msoa_nms['LSOA11CD'].nunique()

41729

In [10]:
msoas = pd.merge(df, msoa_nms, left_on='LSOA Code', right_on='LSOA11CD', how='left')

In [11]:
msoas.shape

(121672, 140)

In [12]:
msoas_by_category = msoas.groupby(['MSOA11CD', 'Major Category'], as_index=False).agg(month_agg)

In [13]:
msoas['Major Category'].unique()

array(['Violence Against the Person', 'Burglary', 'Criminal Damage',
       'Drugs', 'Theft and Handling', 'Other Notifiable Offences',
       'Robbery'], dtype=object)

In [13]:
msoas_by_category.to_csv('processed/Crime_by_MSOA.csv', index=False)
print("Done.")

Done.


In [14]:
pd.pivot_table(msoas_by_category,
              index='MSOA11CD',
              columns='Major Category')

200801                                                  \
Major Category Burglary Criminal Damage Drugs Other Notifiable Offences   
MSOA11CD                                                                  
E02000001             0               0     0                         0   
E02000002             2              28     3                         0   
E02000003            12              20     5                         0   
E02000004             2               9     0                         0   
E02000005             7               7     5                         1   
...                 ...             ...   ...                       ...   
E02006927             6               2     3                         0   
E02006928             2               5     1                         0   
E02006929             8               7     9                         1   
E02006930             5               4     7                         1   
E02006931             9               6     8                         1   

                                                                       \
Major Category Robbery Theft and Handling Violence Against the Person   
MSOA11CD                                                                
E02000001            0                  0                           0   
E02000002            0                 13                          15   
E02000003            4                 26                          17   
E02000004            0                 11                          15   
E02000005            1                 13                          17   
...                ...                ...                         ...   
E02006927            0                 12                           7   
E02006928            4                 10                          10   
E02006929            4                 50                          17   
E02006930            4                 31                           4   
E02006931            6                 37                          19   

                 200802                        ...  201811                     \
Major Category Burglary Criminal Damage Drugs  ... Robbery Theft and Handling   
MSOA11CD                                       ...                              
E02000001             0               0     0  ...     0.0                0.0   
E02000002             5              18     5  ...     3.0               12.0   
E02000003            11               7    11  ...     5.0               21.0   
E02000004             4               3     1  ...     2.0               11.0   
E02000005             5              10     0  ...     4.0               15.0   
...                 ...             ...   ...  ...     ...                ...   
E02006927             7               5     2  ...     2.0               15.0   
E02006928             1               3     0  ...     1.0               44.0   
E02006929            15              11     8  ...     4.0              101.0   
E02006930            14               6     1  ...     2.0               37.0   
E02006931            20               8    11  ...     1.0               47.0   

                                             201812                        \
Major Category Violence Against the Person Burglary Criminal Damage Drugs   
MSOA11CD                                                                    
E02000001                              0.0      0.0             0.0   0.0   
E02000002                             24.0      3.0             5.0   0.0   
E02000003                             37.0      9.0             4.0   1.0   
E02000004                             12.0      3.0             5.0   3.0   
E02000005                             24.0     22.0             3.0   2.0   
...                                    ...      ...             ...   ...   
E02006927                             18.0     16.0             5.0   0.0   
E02006928                             